# Task 5: Finding Multiple Patterns automatically in Sentences

In [3]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
matcher = Matcher(nlp.vocab)

In [6]:
pattern1 = [{"DEP": "nsubj"}, {"DEP": "ROOT"}, {"DEP": "dobj"}]
matcher.add("SubRootObject", [pattern1])

In [7]:
doc = nlp("The big dog chased everybody")

In [8]:
# Find matches
matches = matcher(doc)

In [9]:
# Display dependency visualization
displacy.render(doc, style='dep', options={'compact': True, 'distance': 100})

# 1. What text and dependencies did the above code catch for the sentence “The big dog chased everybody”.

In [10]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Dependency: {}-{}".format(token.text, token.dep))


Matching Sentence:  dog chased everybody
Pattern Type:  SubRootObject
Dependency: dog-429
Dependency: chased-8206900633647566924
Dependency: everybody-416


# 2. Change the sentence to “The big dog chased the cat”. Does the pattern catch the SVO pattern? If not,add another pattern2 to the matcher. The pattern should be DEP: nsubj, DEP: ROOT, DEP: det, DEP: dobj. When done, update matcher.add("SubRootDetObject", [pattern2])


In [11]:
# Change the sentence
doc = nlp("The big dog chased the cat")

In [12]:
pattern2 = [{"DEP": "nsubj"}, {"DEP": "ROOT"}, {"DEP": "det"}, {"DEP": "dobj"}]
matcher.add("SubRootDetObject", [pattern2])


In [13]:
# Find matches
matches = matcher(doc)

In [14]:
# Display dependency visualization
displacy.render(doc, style='dep', options={'compact': True, 'distance': 100})


In [15]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Dependency: {}-{}".format(token.text, token.dep_))

Matching Sentence:  dog chased the cat
Pattern Type:  SubRootDetObject
Dependency: dog-nsubj
Dependency: chased-ROOT
Dependency: the-det
Dependency: cat-dobj


# 3. Now design a third pattern for the sentence “The big dog chased the small cat”.

In [16]:
pattern3 = [{"DEP": "nsubj"}, {"DEP": "ROOT"}, {"DEP": "det"}, {"DEP": "amod"}, {"DEP": "dobj"}]

In [17]:
matcher.add("SubRootDetAmodObject", [pattern3])

In [18]:
doc = nlp("The big dog chased the small cat")

In [19]:
matches = matcher(doc)

In [20]:
displacy.render(doc, style='dep', options={'compact': True, 'distance': 100})


In [21]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Dependency: {}-{}".format(token.text, token.dep))


Matching Sentence:  dog chased the small cat
Pattern Type:  SubRootDetAmodObject
Dependency: dog-429
Dependency: chased-8206900633647566924
Dependency: the-415
Dependency: small-402
Dependency: cat-416


# 1. Design a pattern to identify a noun at least one time. 

In [22]:
# Pattern to identify a noun at least once
pattern4 = [{"POS": "NOUN", "OP": "+"}]

In [23]:
# Finding matches for a sentence
matches = matcher(nlp("The big cat chased the small dog"))

In [24]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Token: {}, POS: {}".format(token.text, token.pos_))


Matching Sentence:  dog chased the small cat
Pattern Type:  SubRootDetAmodObject
Token: dog, POS: NOUN
Token: chased, POS: VERB
Token: the, POS: DET
Token: small, POS: ADJ
Token: cat, POS: NOUN


# 2. Design a pattern to identify a noun of length >= 10 characters.

In [25]:
# Pattern to identify a noun of length >= 10 characters
pattern5 = [{"POS": "NOUN", "LENGTH": {">=": 10}}]

In [26]:
# Finding matches for a sentence
matches = matcher(nlp("The elephant chased the small cat"))

In [27]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Token: {}, POS: {}".format(token.text, token.pos_))


Matching Sentence:  big dog chased the small
Pattern Type:  SubRootDetAmodObject
Token: big, POS: ADJ
Token: dog, POS: NOUN
Token: chased, POS: VERB
Token: the, POS: DET
Token: small, POS: ADJ


# 3. Design a pattern to identify vulgar language (Hint: you will need usage of IN, or NOT_IN).


In [28]:
pattern6 = [{"LOWER": {"IN": ["swear_word1", "swear_word2"]}}]

In [29]:
matches = matcher(doc)

In [30]:
for pattern_id, start, end in matches:
    print("Matching Sentence: ", doc[start:end])
    print("Pattern Type: ", doc.vocab.strings[pattern_id])
    for token in doc[start:end]:
        print("Token: {}, LOWER: {}".format(token.text, token.lower_))


Matching Sentence:  dog chased the small cat
Pattern Type:  SubRootDetAmodObject
Token: dog, LOWER: dog
Token: chased, LOWER: chased
Token: the, LOWER: the
Token: small, LOWER: small
Token: cat, LOWER: cat


# Task 6: Getting Replies

In [31]:
def utterance(msg):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(msg)
    matcher = Matcher(nlp.vocab)

    # Greeting pattern
    pattern1 = [{"LEMMA": {"IN": ["salam", "assalam", "hi", "hello"]}}]
    matcher.add("greeting", [pattern1])

    # Ordering pizza pattern
    pattern2 = [{"LOWER": "i"}, {"LOWER": "would"}, {"LOWER": "like"}, {"LOWER": "to"}, {"LOWER": "order"}, {"LOWER": "a"},
                {"LOWER": "pizza"}]
    matcher.add("ordering_pizza", [pattern2])

    # Complaint pattern
    pattern3 = [{"LOWER": "i"}, {"LOWER": "would"}, {"LOWER": "like"}, {"LOWER": "to"}, {"LOWER": "complain"},
                {"LOWER": "about"}, {"LOWER": "an"}, {"LOWER": "order"}]
    matcher.add("complaint", [pattern3])

    matches = matcher(doc)

    if len(matches) == 0:
        print('Please rephrase your request. Be as specific as possible!')
        return

    for pattern_id, start, end in matches:
        if doc.vocab.strings[pattern_id] == "greeting":
            return "Welcome to Pizza ordering system"
        elif doc.vocab.strings[pattern_id] == "ordering_pizza":
            return "Sure! What type of pizza would you like to order?"
        elif doc.vocab.strings[pattern_id] == "complaint":
            return "I'm sorry to hear about your issue. Please provide details so I can assist you."



In [32]:
# Interaction loop
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    else:
        print("Bot:", utterance(message))

You: hi
Bot: Welcome to Pizza ordering system
You: i would like to order a pizza
Bot: Sure! What type of pizza would you like to order?
You: i would like to complain about an order
Bot: I'm sorry to hear about your issue. Please provide details so I can assist you.
You: quit
